In [1]:
%env CUDA_VISIBLE_DEVICES = '7'

env: CUDA_VISIBLE_DEVICES='7'


In [ ]:
from feature_analysis import *

In [ ]:
sys.path.append('model_opts')
from feature_extraction import *
from model_options import *

In [4]:
imageset = 'oasis'
response_data = load_response_data(imageset, average = False)
image_data = load_image_data(imageset)

In [5]:
model_string = 'ViT-L/14_clip'
model_options = get_model_options()
model_name = model_options[model_string]['model_name']
train_type = model_options[model_string]['train_type']
model_call = model_options[model_string]['call']

model = eval(model_call)
model = model.eval()
if torch.cuda.is_available():
    model = model.cuda()
    
image_transforms = get_recommended_transforms(model_string)

In [6]:
stimulus_loader = DataLoader(dataset=StimulusSet(image_data.image_path, image_transforms), batch_size=64)

In [7]:
target_layers = pd.read_csv('superlative_layers.csv').set_index('model_string').to_dict(orient='index')
target_layer = target_layers[model_string]['model_layer']

In [8]:
stimulus_features = get_all_feature_maps(model, stimulus_loader, numpy=False,
                                         layers_to_retain = [target_layer])

Feature Extraction (Batch):   0%|          | 0/15 [00:00<?, ?it/s]

In [9]:
def treves_rolls(x):
    if isinstance(x, np.ndarray):
        return ((np.sum(x / x.shape[0]))**2 / np.sum(x**2 / x.shape[0]))
    if isinstance(x, torch.Tensor):
        return ((torch.sum(x / x.shape[0]))**2 / torch.sum(x**2 / x.shape[0]))

In [10]:
#source: https://tntorch.readthedocs.io/en/latest/_modules/metrics.html

def torch_skewness(x):
    return torch.mean(((x - torch.mean(x))/torch.std(x))**3)

def torch_kurtosis(x, fisher=True):
    return torch.mean(((x-torch.mean(x))/torch.std(x))**4) - fisher*3

def torch_frobnorm(x):
    return torch.sqrt(torch.clamp(torch.dot(x,x), min=0))

In [13]:
metric_dictlist = []
for model_layer_index, model_layer in enumerate(tqdm(stimulus_features)):
    target_map = stimulus_features[model_layer]
    for target_i, target_activity in enumerate(target_map):
        image_name = image_data.image_name.iloc[target_i]
        
        mean_activity = target_activity.mean().item()
        mean_absolute = target_activity.abs().mean().item()
        max_activity = target_activity.max().item()
        min_activity = target_activity.min().item()
        var_activity = target_activity.std().item()
        var_absolute = target_activity.abs().std().item()
        sparseness = treves_rolls(target_activity).item()
        skewness = torch_skewness(target_activity.abs()).item()
        kurtosis = torch_kurtosis(target_activity.abs()).item()
        frobenius = torch_frobnorm(target_activity.abs()).item()
        activity_range = max_activity - min_activity
        
        metric_dictlist.append({
            'image': image_name, 
            'model': model_name,
            'train_type': train_type,
            'model_layer': model_layer, 
            'model_layer_index': model_layer_index,
            'mean_absolute': mean_absolute,
            'mean_activity': mean_activity,
            'var_activity': var_activity,
            'var_absolute': var_absolute,
            'max_activity': max_activity,
            'min_activity': min_activity,
            'range': activity_range,
            'sparseness': sparseness,
            'skewness': skewness,
            'kurtosis': kurtosis,
            'frobenius': frobenius,
        })
        
metric_data_raw = pd.DataFrame(metric_dictlist)

  0%|          | 0/2 [00:00<?, ?it/s]

In [14]:
metric_data_raw[['mean_activity','var_activity','max_activity','min_activity',
                 'range','sparseness','kurtosis','frobenius', 'mean_absolute','var_absolute']].corr()

,mean_activity,var_activity,max_activity,min_activity,range,sparseness,kurtosis,frobenius,mean_absolute,var_absolute
mean_activity,1.000000,-0.178176,-0.888929,0.993511,-0.969233,0.994691,-0.787801,-0.989987,0.972760,-0.877620
var_activity,-0.178176,1.000000,0.355764,-0.211399,0.284284,-0.216435,0.554901,0.294439,-0.269451,0.603209
max_activity,-0.888929,0.355764,1.000000,-0.903387,0.971115,-0.908793,0.878749,0.922135,-0.911255,0.894406
min_activity,0.993511,-0.211399,-0.903387,1.000000,-0.979616,0.997041,-0.809965,-0.991203,0.973706,-0.893276
range,-0.969233,0.284284,0.971115,-0.979616,1.000000,-0.980502,0.862333,0.983503,-0.968671,0.916024
sparseness,0.994691,-0.216435,-0.908793,0.997041,-0.980502,1.000000,-0.808549,-0.993706,0.973145,-0.893645
kurtosis,-0.787801,0.554901,0.878749,-0.809965,0.862333,-0.808549,1.000000,0.839942,-0.871634,0.923822
frobenius,-0.989987,0.294439,0.922135,-0.991203,0.983503,-0.993706,0.839942,1.000000,-0.979440,0.926980
mean_absolute,0.972760,-0.269451,-0.911255,0.973706,-0.968671,0.973145,-0.871634,-0.979440,1.000000,-0.927911
var_absolute,-0.877620,0.603209,0.894406,-0.893276,0.916024,-0.893645,0.923822,0.926980,-0.927911,1.000000


In [15]:
response_data = {'oasis': load_response_data('oasis')}

In [19]:
def process_metric_data(metric_data, orient='wide'):
    metric_data['dataset'] = imageset
    if 'image' in metric_data.columns:
        metric_data = metric_data.rename(columns={'image': 'image_name'})
    
    data_wide = pd.merge(metric_data, response_data[imageset], on = 'image_name')
    data_wide['model_layer_depth'] = (data_wide['model_layer_index'] / 
                                      data_wide['model_layer'].nunique())
    
    id_columns = ['dataset','image_name','image_type','model','train_type',
                  'model_layer','model_layer_index','model_layer_depth']
    measurement_columns = [col for col in data_wide.columns 
                           if col in ['arousal','beauty','valence']]
    
    analysis_columns = [col for col in data_wide.columns 
                        if col not in id_columns + measurement_columns]
    
    data_wide = data_wide[id_columns + measurement_columns + analysis_columns]
    data_wide = pd.melt(data_wide, id_vars=id_columns + analysis_columns, 
                        var_name = 'measurement', value_name='rating')
    
    data_long = pd.melt(data_wide, id_vars=id_columns + ['measurement', 'rating'], 
                        var_name = 'metric', value_name='value')
    
    if orient == 'wide':
        return(data_wide)
    if orient == 'long':
        return(data_long)
    
def process_corr_data(data_wide, include_combo = True, orient='long'):
    model_layers = data_wide['model_layer'].unique().tolist()
    
    id_columns = ['model','train_type','dataset','image_type','model_layer',
                  'model_layer_index','model_layer_depth', 'measurement']
    
    corr_data_wide = (data_wide.groupby(id_columns).corrwith(data_wide['rating'])
                      .reset_index().drop('rating',axis = 1))
    
    if include_combo:
        
        id_columns_ = [col for col in id_columns if col != 'image_type']
        
        corr_data_wide_ = (data_wide.groupby(id_columns_).corrwith(data_wide['rating'])
                           .reset_index().drop('rating',axis = 1))
        corr_data_wide_['image_type'] = 'Combo'
        
        corr_data_wide = pd.concat([corr_data_wide, corr_data_wide_])
        
    
    corr_data_long = pd.melt(corr_data_wide, id_vars = id_columns, 
                             var_name = 'metric', value_name='corr')
        
    if orient == 'wide':
        return(corr_data_wide)
    if orient == 'long':
        return(corr_data_long)


In [20]:
metric_data = process_metric_data(metric_data_raw)

In [21]:
corr_data = process_corr_data(metric_data)
corr_data['corr_abs'] = abs(corr_data['corr'])

In [30]:
corr_data = corr_data[corr_data['model_layer'] == 'LayerNorm-50']

In [31]:
corr_data.groupby(['metric'])['corr'].mean().reset_index()

,metric,corr
0,frobenius,0.037576
1,kurtosis,0.061997
2,max_activity,0.045373
3,mean_absolute,-0.073688
4,mean_activity,0.028064
5,min_activity,-0.085681
6,range,0.094804
7,skewness,0.068877
8,sparseness,-0.011416
9,var_absolute,0.102320


In [33]:
(corr_data[(corr_data['measurement'] == 'beauty')]
 .groupby(['metric'])['corr_abs'].mean().reset_index().sort_values(by='corr_abs'))

,metric,corr_abs
0,frobenius,0.093013
10,var_activity,0.093548
5,min_activity,0.104890
8,sparseness,0.108713
4,mean_activity,0.123655
1,kurtosis,0.123934
7,skewness,0.164673
2,max_activity,0.183709
6,range,0.209772
9,var_absolute,0.214068


In [45]:
(corr_data[(corr_data['measurement'] == 'beauty') & (corr_data['image_type'] == 'Scene')]
 .groupby(['metric'])['corr_abs'].mean().reset_index().sort_values(by='corr_abs'))

,metric,corr_abs
8,sparseness,0.057763
10,var_activity,0.075277
0,frobenius,0.078855
1,kurtosis,0.151339
4,mean_activity,0.170191
5,min_activity,0.183185
7,skewness,0.229123
3,mean_absolute,0.360119
6,range,0.386830
2,max_activity,0.392457


In [24]:
max_transform(corr_data, group_vars = ['measurement', 'image_type', 'metric'],
              measure_var = 'corr').groupby(['metric'])['corr'].mean().reset_index()

,metric,corr
0,frobenius,0.206399
1,kurtosis,0.147428
2,max_activity,0.179856
3,mean_absolute,0.057358
4,mean_activity,0.126466
5,min_activity,-0.020278
6,range,0.185246
7,skewness,0.160121
8,sparseness,0.032371
9,var_absolute,0.176480


In [46]:
max_transform(corr_data[(corr_data['measurement'] == 'beauty') & (corr_data['image_type'] == 'Scene')],
              group_vars = ['metric'], measure_var = 'corr').groupby(['metric'])['corr'].mean().reset_index()

,metric,corr
0,frobenius,0.078855
1,kurtosis,0.151339
2,max_activity,0.392457
3,mean_absolute,-0.360119
4,mean_activity,0.170191
5,min_activity,-0.183185
6,range,0.386830
7,skewness,0.229123
8,sparseness,0.057763
9,var_absolute,0.424799


In [50]:
import numba

NAN = float("nan")

@numba.njit(nogil=True)
def _any_nans(a):
    for x in a:
        if np.isnan(x): return True
    return False

@numba.jit
def any_nans(a):
    if not a.dtype.kind=='f': return False
    return _any_nans(a.flat)

import pingouin as pg

In [48]:
target_metrics = ['mean_activity', 'mean_absolute', 'var_activity', 'var_absolute', 'max_activity', 'min_activity',
                  'range', 'sparseness', 'skewness', 'kurtosis', 'frobenius']

results_dictlist = []
data_wide = metric_data
model_layers = data_wide['model_layer'].unique()
for measurement in data_wide['measurement'].unique():
        for image_type in data_wide['image_type'].unique():
            for metric in target_metrics:
                data_i = data_wide[(data_wide['image_type'] == image_type) & 
                                   (data_wide['measurement'] == measurement)]
                y = data_i[(data_i['model_layer']==model_layers[0])]['rating'].to_numpy()
                X = np.stack([data_i[(data_i['model_layer']==model_layer)][metric].to_numpy() 
                              for model_layer in model_layers], axis = 1)

                actual_max = max([abs(pearsonr(x, y)[0]) for x in X.transpose()
                                  if not any_nans(x)])

                permuted_max_corrs = []
                for i in range(1000):
                    permuted_corrs = [abs(pearsonr(np.random.permutation(x), y)[0]) 
                                      for x in X.transpose() if not any_nans(x)]
                    permuted_max_corrs.append(max(permuted_corrs))

                permuted_lqt = np.quantile(permuted_max_corrs, 0.025)
                permuted_uqt = np.quantile(permuted_max_corrs, 0.975)
                permuted_pvalue = (len([corr for corr in permuted_max_corrs if corr >= actual_max])) / 1000

                results_dictlist.append({'model': model_name, 'train_type': train_type, 
                                         'dataset': 'oasis', 'image_type': image_type, 
                                         'metric': metric, 'measurement': measurement,
                                         'model_depth': len(model_layers),
                                         'corr_max_score': actual_max,
                                         'corr_lower_ci': permuted_lqt,
                                         'corr_upper_ci': permuted_uqt,
                                         'corr_p_value': permuted_pvalue})


metric_permutations = pd.DataFrame(results_dictlist)

In [51]:
metric_permutations['corr_p_adj'] = pg.multicomp(metric_permutations['corr_p_value'].to_numpy(), 
                                                 alpha = 0.05, method = 'fdr')[1]

In [52]:
metric_permutations[['measurement','image_type']].value_counts()

measurement  image_type
arousal      Animal        11
             Object        11
             Person        11
             Scene         11
beauty       Animal        11
             Object        11
             Person        11
             Scene         11
valence      Animal        11
             Object        11
             Person        11
             Scene         11
dtype: int64

In [53]:
metric_permutations.query('corr_p_value < 0.05')[['measurement','image_type']].value_counts()

measurement  image_type
arousal      Object        11
             Scene         11
beauty       Scene         10
valence      Scene          9
             Object         8
arousal      Person         7
beauty       Animal         7
             Object         7
             Person         7
valence      Animal         5
             Person         3
arousal      Animal         1
dtype: int64

In [55]:
metric_permutations.query('corr_p_value < 0.05')[['metric']].value_counts()

metric       
mean_absolute    11
max_activity      9
min_activity      9
range             9
kurtosis          8
skewness          8
frobenius         7
mean_activity     7
var_activity      7
var_absolute      6
sparseness        5
dtype: int64

In [54]:
metric_permutations[(metric_permutations['measurement'] == 'beauty') & 
                    (metric_permutations['image_type'] == 'Scene')] 

,model,train_type,dataset,image_type,metric,measurement,model_depth,corr_max_score,corr_lower_ci,corr_upper_ci,corr_p_value,corr_p_adj
121,ViT-L/14,clip,oasis,Scene,mean_activity,beauty,2,0.170191,0.014483,0.171334,0.026,0.045158
122,ViT-L/14,clip,oasis,Scene,mean_absolute,beauty,2,0.360119,0.014183,0.166285,0.000,0.000000
123,ViT-L/14,clip,oasis,Scene,var_activity,beauty,2,0.476059,0.013753,0.166087,0.000,0.000000
124,ViT-L/14,clip,oasis,Scene,var_absolute,beauty,2,0.498141,0.013720,0.171347,0.000,0.000000
125,ViT-L/14,clip,oasis,Scene,max_activity,beauty,2,0.392457,0.013888,0.163503,0.000,0.000000
126,ViT-L/14,clip,oasis,Scene,min_activity,beauty,2,0.183185,0.014624,0.172045,0.016,0.029746
127,ViT-L/14,clip,oasis,Scene,range,beauty,2,0.386830,0.011666,0.164330,0.000,0.000000
128,ViT-L/14,clip,oasis,Scene,sparseness,beauty,2,0.057763,0.014786,0.162606,0.621,0.650571
129,ViT-L/14,clip,oasis,Scene,skewness,beauty,2,0.421396,0.012097,0.172124,0.000,0.000000
130,ViT-L/14,clip,oasis,Scene,kurtosis,beauty,2,0.350817,0.013507,0.168444,0.000,0.000000
